# Chronos Bolt Base - Fine Tuning, No covariates

Document: https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-chronos.html#fine-tuning

In [8]:
import datetime

import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from sklearn.metrics import root_mean_squared_error

In [2]:
# Experiment設定
EXPERIMENT_NAME = 'ChronosBolt_FT'
MODEL = "bolt_base"
FORECAST_HORIZON = 6

In [3]:
df_train = pd.read_csv('../data/walmart_ts_6_fcst_grp_train.csv')
df_test = pd.read_csv('../data/walmart_ts_6_fcst_grp_test.csv')
print(df_train.shape, df_test.shape)
#df_train.head()

# 日付カラムを型変換する場合
#df_train['Date'] = pd.to_datetime(df_train['Date'])
#df_test['Date'] = pd.to_datetime(df_test['Date'])

# 単一のグループIDを作成
df_train['Store_Dept'] = [str(i)+'_'+str(j) for i,j in zip(df_train['Store'], df_train['Dept'])]
df_test['Store_Dept'] = [str(i)+'_'+str(j) for i,j in zip(df_test['Store'], df_test['Dept'])]

# 利用カラムのみを選択
df_train = df_train[['Store_Dept','Date','Weekly_Sales']]
df_test = df_test[['Store_Dept','Date','Weekly_Sales']]
print(df_train.shape, df_test.shape)
df_train.head()

(702, 11) (36, 11)
(702, 3) (36, 3)


,Store_Dept,Date,Weekly_Sales
0,4_4,2010/2/5,59554.57
1,4_4,2010/2/12,54069.82
2,4_4,2010/2/19,53939.17
3,4_4,2010/2/26,54687.08
4,4_4,2010/3/5,56959.02


In [4]:
# TimeSeriesDataFrameへ変換
train_data = TimeSeriesDataFrame.from_data_frame(
    df_train,
    id_column='Store_Dept',
    timestamp_column="Date"
)
print(type(train_data))
train_data.head()

<class 'autogluon.timeseries.dataset.ts_dataframe.TimeSeriesDataFrame'>


Weekly_Sales
item_id timestamp               
4_4     2010-02-05      59554.57
        2010-02-12      54069.82
        2010-02-19      53939.17
        2010-02-26      54687.08
        2010-03-05      56959.02

In [13]:
%%time

log_dir = EXPERIMENT_NAME +'_'+ datetime.datetime.now().strftime('%Y-%m-%d_%Hh%Mm%Ss')
print(log_dir)

predictor = TimeSeriesPredictor(
    target="Weekly_Sales",
    eval_metric="RMSE",
    prediction_length=FORECAST_HORIZON,
    path="./tmp/"+log_dir,
).fit(
    train_data=train_data,
    hyperparameters={
        "Chronos": {"model_path": MODEL, 
                    "fine_tune": True, 
                    "ag_args": {"name_suffix": "FineTuned"}},
    },
    time_limit=300,  # time limit in seconds
    enable_ensemble=False,
)
predictor

Beginning AutoGluon training... Time limit = 300s
AutoGluon will save models to '/Users/YShimada-MBP16/Documents/Code_Github/DL_for_ImageData_and_Finetuning/TimeSeries_WM_simple/code/tmp/ChronosBolt_FT_2025-02-27_10h23m12s'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.14
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:22:00 PST 2025; root:xnu-11215.81.4~3/RELEASE_X86_64
CPU Count:          16
GPU Count:          0
Memory Avail:       14.85 GB / 32.00 GB (46.4%)
Disk Space Avail:   1435.59 GB / 1863.39 GB (77.0%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': RMSE,
 'hyperparameters': {'Chronos': {'ag_args': {'name_suffix': 'FineTuned'},
                                 'fine_tune': True,
                                 'model_path': 'bolt_base'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_leve

ChronosBolt_FT_2025-02-27_10h23m12s


	Fine-tuning on the CPU detected. We recommend using a GPU for faster fine-tuning of Chronos.
	Saving fine-tuned model to /Users/YShimada-MBP16/Documents/Code_Github/DL_for_ImageData_and_Finetuning/TimeSeries_WM_simple/code/tmp/ChronosBolt_FT_2025-02-27_10h23m12s/models/ChronosFineTuned[bolt_base]/W0/fine-tuned-ckpt
	-2391.8344    = Validation score (-RMSE)
	243.49  s     = Training runtime
	0.17    s     = Validation (prediction) runtime
Training complete. Models trained: ['ChronosFineTuned[bolt_base]']
Total runtime: 243.68 s
Best model: ChronosFineTuned[bolt_base]
Best model score: -2391.8344


CPU times: user 30min 44s, sys: 1min 4s, total: 31min 48s
Wall time: 4min 3s


In [14]:
# 与えられたデータの最後の日付から、prediction_length分を予測
predictions = predictor.predict(train_data)
predictions

Model not specified in predict, will default to the model with the best validation score: ChronosFineTuned[bolt_base]


mean           0.1           0.2           0.3  \
item_id timestamp                                                            
4_4     2012-05-04  60160.710938  56614.203125  57710.664062  58501.574219   
        2012-05-11  60837.765625  57702.574219  58618.117188  59367.933594   
        2012-05-18  61010.031250  57863.558594  58814.382812  59590.738281   
        2012-05-25  61021.304688  58061.941406  58948.253906  59647.691406   
        2012-06-01  62011.417969  58843.730469  59807.449219  60567.800781   
        2012-06-08  60645.015625  57846.382812  58720.097656  59393.691406   
8_13    2012-05-04  24687.005859  23363.060547  23799.777344  24111.806641   
        2012-05-11  24615.462891  23378.964844  23796.656250  24099.718750   
        2012-05-18  24604.689453  23307.408203  23750.718750  24074.722656   
        2012-05-25  24726.457031  23427.128906  23895.580078  24212.195312   
        2012-06-01  24949.591797  23597.197266  24061.046875  24393.708984   
        2012-06-08  24680.679688  23385.351562  23843.248047  24165.671875   
1_13    2012-05-04  42855.335938  40346.683594  41229.867188  41793.003906   
        2012-05-11  43026.476562  40784.472656  41538.890625  42102.242188   
        2012-05-18  41695.656250  39538.367188  40327.437500  40860.632812   
        2012-05-25  42125.207031  39827.730469  40629.433594  41186.445312   
        2012-06-01  43873.941406  41193.777344  42107.718750  42760.531250   
        2012-06-08  44546.398438  41797.636719  42832.164062  43479.636719   
13_8    2012-05-04  35417.714844  33808.691406  34337.437500  34725.023438   
        2012-05-11  34897.316406  33441.457031  33904.339844  34269.800781   
        2012-05-18  34388.660156  32926.613281  33421.296875  33794.660156   
        2012-05-25  34933.871094  33373.652344  33905.097656  34289.011719   
        2012-06-01  36115.273438  34191.546875  34841.019531  35334.921875   
        2012-06-08  37686.437500  35331.656250  36202.414062  36777.457031   
19_8    2012-05-04  42226.148438  40231.335938  40915.324219  41391.117188   
        2012-05-11  41752.464844  39940.058594  40553.605469  40999.085938   
        2012-05-18  41170.234375  39414.824219  40011.433594  40445.097656   
        2012-05-25  41411.789062  39660.949219  40284.652344  40704.601562   
        2012-06-01  41862.988281  39892.777344  40565.546875  41054.691406   
        2012-06-08  43391.222656  40977.445312  41861.890625  42448.753906   
31_13   2012-05-04  43539.796875  41709.199219  42324.644531  42735.339844   
        2012-05-11  44098.417969  42249.316406  42836.484375  43297.980469   
        2012-05-18  46619.792969  44199.207031  45021.523438  45633.363281   
        2012-05-25  46246.222656  44167.789062  44888.835938  45367.367188   
        2012-06-01  45377.843750  43542.007812  44132.480469  44584.453125   
        2012-06-08  44996.062500  43218.054688  43859.078125  44290.609375   

                             0.4           0.5           0.6           0.7  \
item_id timestamp                                                            
4_4     2012-05-04  59318.980469  60160.710938  61001.585938  61951.273438   
        2012-05-11  60090.035156  60837.765625  61641.257812  62534.054688   
        2012-05-18  60294.664062  61010.031250  61709.460938  62511.914062   
        2012-05-25  60297.085938  61021.304688  61727.312500  62484.769531   
        2012-06-01  61275.222656  62011.417969  62756.960938  63509.500000   
        2012-06-08  59958.894531  60645.015625  61359.078125  62105.468750   
8_13    2012-05-04  24399.871094  24687.005859  24971.357422  25287.890625   
        2012-05-11  24360.873047  24615.462891  24877.816406  25184.119141   
        2012-05-18  24350.986328  24604.689453  24861.361328  25153.050781   
        2012-05-25  24472.000000  24726.457031  24984.783203  25277.746094   
        2012-06-01  24677.279297  24949.591797  25218.416016  25510.289062   
        2012-06-08  24420.111328  24680.679688  2

In [15]:
df_pred = predictions.to_data_frame().reset_index()

print('RMSE: ', root_mean_squared_error(y_true=df_test['Weekly_Sales'], y_pred=df_pred['mean']))

df_pred.to_csv('../Chronos_results/{}.csv'.format(EXPERIMENT_NAME), index=False)

RMSE:  1989.2078543564032
